In [1]:
import torch
import os 
import random 
from natsort import natsorted

In [2]:
data_directory = "data/task_from_graph_tensor/"
data_files = os.listdir(data_directory)
data_files = natsorted(data_files)
print(f"Lenght of data_files: {len(data_files)}")
print(f"First 10 data_fiels {data_files[:10]}")

list_of_iso_graphs = [] # List of all unique graphs
list_of_same_graphs = [] # Graph having the same structure iso
for idx, data_file in enumerate(data_files):
    data = torch.load(data_directory + data_file)
    if idx % 100 == 0 and idx != 0:
        list_of_iso_graphs.append(list_of_same_graphs.copy())
        list_of_same_graphs.clear()
        list_of_same_graphs.append(data)
    else: 
        list_of_same_graphs.append(data)

print(f"Length of list_of_iso_graphs: {len(list_of_iso_graphs)}")
print(f"length of each iso list : {len(list_of_iso_graphs[0])}")

Lenght of data_files: 81200
First 10 data_fiels ['graph_0.pt', 'graph_1.pt', 'graph_2.pt', 'graph_3.pt', 'graph_4.pt', 'graph_5.pt', 'graph_6.pt', 'graph_7.pt', 'graph_8.pt', 'graph_9.pt']
Length of list_of_iso_graphs: 811
length of each iso list : 100


In [4]:
# Splitting Train and Test
random.seed(35)
num_test_data = 20
list_of_iso_test = random.sample(list_of_iso_graphs, num_test_data)
print(f"Length of test data: {len(list_of_iso_test)}")
list_of_iso_train = [item for item in list_of_iso_graphs if item not in list_of_iso_test]
print(f"Lenght of train data: {len(list_of_iso_train)}")

Length of test data: 10
Lenght of train data: 801


In [27]:
import numpy as np

def filter_data(list_of_iso):
    filtered_iso_list = []
    for idx, iso_list in enumerate(list_of_iso):
        latency_list = []
        without_outlier_list = []
        for data in iso_list:
            latency_list.append(data.y)
        latency_array = np.array(latency_list)

        q1 = np.percentile(latency_array, 25)
        q2 = np.median(latency_array)
        q3 = np.percentile(latency_array, 75)

        iqr = q3 - q1

        box_min_val = q1 - 1.5 * iqr
        box_max_val = q3 + 1.5 * iqr
        
        global_min = np.min(latency_array)
        global_max = np.max(latency_array)

        for data in iso_list:
            # if data.y > box_min_val and data.y < box_max_val: # Removing outliers
            # if data.y > q1 and data.y < q3: # Removing first and last quartile
            # if data.y >= global_min and data.y <= q2: # Removing first and last quartile
                # without_outlier_list.append(data)
            without_outlier_list.append(data) # No filtering
        filtered_iso_list.append(without_outlier_list)
    return filtered_iso_list

filtered_iso_train = filter_data(list_of_iso_train)
print(f"Filtered_iso_train: {len(filtered_iso_train)}")
filtered_iso_test = filter_data(list_of_iso_test)
print(f"Filtered_iso_test: {len(filtered_iso_test)}")

Filtered_iso_train: 801
Filtered_iso_test: 10


In [28]:
for idx, iso_list in enumerate(filtered_iso_train):
    print(f"Length of iso_list {idx}: {len(iso_list)}")

Length of iso_list 0: 57
Length of iso_list 1: 50
Length of iso_list 2: 51
Length of iso_list 3: 50
Length of iso_list 4: 51
Length of iso_list 5: 54
Length of iso_list 6: 50
Length of iso_list 7: 50
Length of iso_list 8: 53
Length of iso_list 9: 52
Length of iso_list 10: 54
Length of iso_list 11: 50
Length of iso_list 12: 50
Length of iso_list 13: 51
Length of iso_list 14: 50
Length of iso_list 15: 50
Length of iso_list 16: 52
Length of iso_list 17: 53
Length of iso_list 18: 50
Length of iso_list 19: 50
Length of iso_list 20: 50
Length of iso_list 21: 50
Length of iso_list 22: 50
Length of iso_list 23: 58
Length of iso_list 24: 58
Length of iso_list 25: 53
Length of iso_list 26: 51
Length of iso_list 27: 50
Length of iso_list 28: 52
Length of iso_list 29: 52
Length of iso_list 30: 54
Length of iso_list 31: 51
Length of iso_list 32: 50
Length of iso_list 33: 50
Length of iso_list 34: 54
Length of iso_list 35: 50
Length of iso_list 36: 51
Length of iso_list 37: 51
Length of iso_list 38:

In [29]:
# Dont run for filtered stuff
import pickle 

with open("data/task_from_graph_test.pkl", "wb") as f:
    pickle.dump(filtered_iso_test, f)

In [26]:
data_directory = "data/task_from_graph_upto_q2_train/"
count = 0
for _, same_graph_list in enumerate(filtered_iso_train):
    for idx, data in enumerate(same_graph_list):
        torch.save(data, data_directory + f"{count}.pt")
        count += 1